<a href="https://colab.research.google.com/github/VOX304/SchoolChatbot/blob/main/Preprocessing/Colab_preprocessing/Table_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber python-docx opencv-python numpy vietocr boundbox markdown-table


In [ ]:
!pip install markdown-table


In [ ]:
!apt update
!apt install -y libreoffice


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
30 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
!pip install pytesseract
!pip install pdf2image
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,668 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 134513 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

def is_text_valid(text, threshold=0.5):
    """Check if extracted text is valid or gibberish based on character ratios."""
    non_alpha_chars = sum(1 for char in text if not char.isalnum() and char not in " \n")
    alpha_chars = sum(1 for char in text if char.isalnum())

    if alpha_chars == 0:
        return False  # All non-alphanumeric, probably junk

    non_alpha_ratio = non_alpha_chars / (alpha_chars + non_alpha_chars)
    return non_alpha_ratio < threshold  # If too many non-alphanumeric chars, it's bad

def extract_table_text_based(file_path):
    """Extract table data from PDFs or DOCX files, fallback to OCR if text is bad."""
    tables = []

    if file_path.endswith('.pdf'):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                extracted_text = page.extract_text()

                if not extracted_text or not is_text_valid(extracted_text):
                    print("⚠️ Poor text quality detected. Switching to OCR-based extraction.")
                    tables.extend(extract_table_from_image_pdf(file_path))
                    return tables  # Return early since we used image OCR

                tables.extend(page.extract_tables())

    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        for table in doc.tables:
            rows = [[cell.text for cell in row.cells] for row in table.rows]
            tables.append(rows)

    return tables

def extract_table_from_image_pdf(file_path):
    """Convert PDF pages to images and extract tables using OCR."""
    tables = []
    images = convert_from_path(file_path, dpi=300)  # Convert PDF to images

    for img in images:
        text = pytesseract.image_to_string(img)
        structured_table = reconstruct_table_structure(text, img)
        tables.append(structured_table)

    return tables


In [ ]:
import cv2
import numpy as np

def align_text_to_grid(text, contours):
    """Align OCR-extracted text to a table grid using contour detection."""
    rows = []

    # Sort contours top-to-bottom (to process rows correctly)
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        # Extract text inside this cell
        cell_text = text.strip()
        rows.append(cell_text)

    return [rows]  # Return as a list of lists (table format)


In [ ]:
def reconstruct_table_structure(text, image):
    """Use contour detection & grid alignment to reconstruct table format."""
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    structured_data = align_text_to_grid(text, contours)
    return structured_data


In [ ]:
import os
import time

def convert_doc_to_docx(doc_path):
    """Convert a .doc file to .docx using LibreOffice."""
    output_path = doc_path.rsplit(".", 1)[0] + ".docx"

    if not os.path.exists(doc_path):
        raise FileNotFoundError(f"File not found: {doc_path}")

    os.system(f'soffice --headless --convert-to docx "{doc_path}"')

    # Wait for the file to appear (up to 5 seconds)
    timeout = 7
    while timeout > 0:
        if os.path.exists(output_path):
            return output_path
        time.sleep(1)
        timeout -= 1

    raise FileNotFoundError(f"Conversion failed. No .docx file found at {output_path}")


In [ ]:
def store_in_markdown_file(markdown_tables, output_file="tables.md"):
    """Save extracted tables in a Markdown file."""
    with open(output_file, "w", encoding="utf-8") as f:
        for table in markdown_tables:
            f.write(table + "\n\n")  # Separate tables with a newline
    print(f"Tables saved to {output_file}")


In [ ]:
import pdfplumber
import docx
import cv2
import numpy as np
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

# Load OCR model
config = Cfg.load_config_from_name('vgg_transformer')
config['device'] = 'cpu'  # Change to 'cuda' if GPU available
ocr_model = Predictor(config)

def detect_table_presence(file_path):
    """Determine if a document contains tables and whether they are text-based or image-based."""
    if file_path.endswith('.pdf'):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                if page.extract_tables():
                    return 'text'
        return 'image'
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        for table in doc.tables:
            return 'text'
    return 'image'

def detect_table(image_path):
    """Detect tables using OpenCV."""
    image = cv2.imread(image_path)

    if image is None:
        raise ValueError(f"Error: Could not load image from {image_path}. Check file path and format.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Table bounding boxes
    tables = [cv2.boundingRect(cnt) for cnt in contours]
    return tables

def extract_table_text_based(file_path):
    """Extract table data from text-based PDFs or DOCX files."""
    tables = []
    if file_path.endswith('.pdf'):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                tables.extend(page.extract_tables())
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        for table in doc.tables:
            rows = [[cell.text for cell in row.cells] for row in table.rows]
            tables.append(rows)
    return tables

def apply_ocr_to_image(image):
    """Use VietOCR to recognize text from an image."""
    return ocr_model.predict(image)

def extract_table_from_image(image_path):
    """Apply BoundBox + OCR to extract tables from image-based PDFs."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    tables = detect_table(image)  # Detect table boundaries
    extracted_tables = []

    for table in tables:
        x, y, w, h = table  # Table bounding box
        cropped_table = image[y:y+h, x:x+w]
        text = apply_ocr_to_image(cropped_table)
        structured_table = reconstruct_table_structure(text, cropped_table)
        extracted_tables.append(structured_table)

    return extracted_tables

def reconstruct_table_structure(text, image):
    """Use contour detection & grid alignment to reconstruct table format."""
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    structured_data = align_text_to_grid(text, contours)
    return structured_data

def convert_tables_to_markdown(tables):
    """Convert extracted tables into Markdown format."""
    return [convert_to_markdown(table) for table in tables]

def convert_to_markdown(table_data):
    """Convert extracted table data (list of lists) into Markdown format."""
    if not table_data:
        return ""

    markdown = "| " + " | ".join(table_data[0]) + " |\n"
    markdown += "| " + " | ".join(["---"] * len(table_data[0])) + " |\n"

    for row in table_data[1:]:
        markdown += "| " + " | ".join(row) + " |\n"

    return markdown


def store_in_vectordb(markdown_tables):
    """Store processed Markdown tables into a VectorDB for retrieval."""
    for table in markdown_tables:
        print("Storing table in VectorDB:\n", table)
    # VectorDB storage logic here (FAISS, Pinecone, etc.)

def process_document(file_path):
    """Complete pipeline: Detect table type, extract, convert to Markdown, and store in VectorDB."""
    table_type = detect_table_presence(file_path)

    if table_type == 'text':
        tables = extract_table_text_based(file_path)
    else:
        tables = extract_table_from_image(file_path)

    markdown_tables = convert_tables_to_markdown(tables)

    store_in_markdown_file(markdown_tables, output_file="extracted_tables.md")
    print("Table processing completed.")

# Example usage:
# process_document("example.pdf")


In [ ]:
file_path = "/content/HD1860 công tác xét tuyển đào tạo ĐH, CĐ năm 2024_20252211126.pdf"

res = pdfplumber.open(file_path)
print_result = res.pages[0].extract_text()
print(print_result)
#converted_path = convert_doc_to_docx(file_path)

#print("Converted file path:", converted_path)